In [ ]:
import numpy as np
from utils.extract_topics import RosbagSensorExtractor

The Problem in Simple Terms
Think of it like this:

Your camera takes a photo at 3:00:00 PM
Your LiDAR scans at 3:00:00.05 PM (50 milliseconds later)
During those 50ms, your car/robot moved a little bit
If you just overlay them without accounting for movement, they won't match up

The IMU tells you: "Hey, during those 50ms, you rotated 2 degrees left and moved 10cm forward"
What is Preintegration?
Preintegration is just adding up all the tiny movements the IMU measured.
Think of it like this:

IMU runs at 100 Hz (100 measurements per second)
Between your camera (t=3.00) and LiDAR (t=3.05), you have maybe 5 IMU measurements
Each one says: "you rotated a tiny bit" and "you accelerated a tiny bit"
You integrate (add them up) to get the total change

In [2]:
class IMUPreintegration:
    def __init__(self):
        self.delts_p = np.zeros(3)


In [3]:
if __name__ == "__main__":
    extractor = RosbagSensorExtractor(
        bag_path="dataset/snell_floor_2_ros2_bag",
        camera_topic="/camera_array/cam0/image_raw",
        lidar_topic="/ouster/points",
        imu_topic="/imu/imu_compensated",
    )
    
    extractor.extract()
  


[INFO] Reading bag: dataset/snell_floor_2_ros2_bag
[INFO] Camera topic: /camera_array/cam0/image_raw
[INFO] LiDAR topic:  /ouster/points
[INFO] IMU topic:    /imu/imu_compensated


Extracting: 100%|██████████| 7327/7327 [00:22<00:00, 330.42msg/s]


[INFO] Extraction complete:
  Camera frames: 637
  LiDAR scans:   318
  IMU samples:   6372


In [4]:
print(f"\n[CAM]")
print(f"t_sec: {extractor.camera_frames[0]['timestamp_sec']}")
print(f"frame_id: {extractor.camera_frames[0]['frame_id']}")
print(f"image shape: {extractor.camera_frames[0]['image'].shape}")

print(f"\n[LIDAR]")
print(f"t_sec: {extractor.lidar_scans[0]['timestamp_sec']}")
print(f"frame_id: {extractor.lidar_scans[0]['frame_id']}")
print(f"points shape: {extractor.lidar_scans[0]['points'].shape}")

print(f"\n[IMU]")
print(f"t_sec: {extractor.imu_measurements[0]['timestamp_sec']}")
print(f"frame_id: {extractor.imu_measurements[0]['frame_id']}")
print(f"acc: {extractor.imu_measurements[0]['linear_acceleration']}")
print(f"gyro: {extractor.imu_measurements[0]['angular_velocity']}")


[CAM]
t_sec: 1679007787.4866676
frame_id: cam_0_optical_frame
image shape: (540, 720, 3)

[LIDAR]
t_sec: 1679007787.5265582
frame_id: os_sensor
points shape: (524288, 3)

[IMU]
t_sec: 1679007787.4450188
frame_id: imu
acc: [ -4.6586423   2.3057532 -12.003281 ]
gyro: [-0.00290214 -0.00849772 -0.08366963]


three things to compute

ΔR (Delta Rotation) - How much did you rotate?
Δv (Delta Velocity) - How much did your speed change?
Δp (Delta Position) - How much did you move?

Step 1: Get Your Synchronized Data
First, find which camera, LiDAR, and IMU data go together:

In [ ]:
# You have camera at time t
camera_frame = extractor.camera_frames[0]
t_camera = camera_frame['timestamp_sec']

# Find closest LiDAR scan (this will be at t + delta_t)
# Which LiDAR scan is closest to this camera time
lidar_scan = # find closest one

t_lidar = lidar_scan['timestamp_sec']
delta_t = t_lidar - t_camera  # This is your time gap

# Get ALL IMU measurements between camera and LiDAR
imu_between = # all IMU data where t_camera <= t <= t_lidar

Step 2: Initialize Your Preintegration
Before you start adding up movements, start from zero:

In [ ]:
# Start with no change
delta_rotation = np.eye(3)  # Identity matrix (no rotation)
delta_velocity = np.zeros(3)  # Not moving
delta_position = np.zeros(3)  # Haven't moved


### Step 3: Loop Through Each IMU Measurement

# you have 5 IMU measurements between camera and LiDAR:

# Camera ----[IMU1]----[IMU2]----[IMU3]----[IMU4]----[IMU5]---- LiDAR
#   t=3.00                                                    t=3.05

In [ ]:
# for each measurement we do
for i in range(len(imu_between)):
    current_imu = imu_between[i]
    
    # Get the measurements
    gyro = current_imu['angular_velocity']  # tells  rotation rate
    acc = current_imu['linear_acceleration']  # tells acceleration
    
    # How much time passed since last measurement
    if i == 0:
        dt = current_imu['timestamp_sec'] - t_camera
    else:
        dt = current_imu['timestamp_sec'] - imu_between[i-1]['timestamp_sec']
    
    # Now update your deltas

Step 4: The Integration Math (Simplified)
For each IMU measurement, you update three things:

In [ ]:
# A) Update Rotation:

# Gyroscope tells you: "rotating at X rad/s"
# Over dt seconds, that's: X * dt total rotation
angle_change = gyro * dt  # This is a 3D vector [roll, pitch, yaw] change

# Convert this small rotation to a rotation matrix and multiply
delta_rotation = delta_rotation @ rotation_from_vector(angle_change)

In [ ]:
# B) Update Velocity:

# Acceleration needs to be in GLOBAL frame, not sensor frame
# That's why we rotate it by current delta_rotation
acc_global = delta_rotation @ acc

# Remove gravity (gravity is always pulling down at 9.81 m/s²)
acc_global -= np.array([0, 0, 9.81])

# Velocity = old velocity + acceleration * time
delta_velocity += acc_global * dt


In [ ]:
# C) Update Position:

# Position = old position + velocity * time + 0.5 * acceleration * time²
delta_position += delta_velocity * dt + 0.5 * (delta_rotation @ acc) * dt**2

Step 5: Final Result
After looping through all IMU measurements, you have:

delta_rotation - a 3x3 rotation matrix
delta_velocity - a 3D vector [vx, vy, vz]
delta_position - a 3D vector [x, y, z]

This tells you: "Between camera time and LiDAR time, the sensor moved THIS much"

Why Does This Help?
Now you can transform the LiDAR points backwards in time to match the camera:

In [ ]:
# LiDAR point at time t+Δt
lidar_point = np.array([x, y, z])

# Move it back to camera time t
lidar_point_at_camera_time = delta_rotation.T @ (lidar_point - delta_position)

# Now it aligns with the camera image!

In [ ]:
# 1. Pick a camera frame at time t_cam
camera_frame = extractor.camera_frames[i]
t_cam = camera_frame['timestamp_sec']

# 2. Find closest LiDAR scan at time t_lidar
t_lidar = find_closest_lidar(t_cam)

# 3. Get ALL IMU between these times
imu_measurements = get_imu_between(t_cam, t_lidar)

# 4. Integrate ALL of them to get Tv
Tv = preintegrate_imu(imu_measurements)

# 5. Later: use line features to correct errors in Tv

1. Why Rotate Acceleration? (You're asking the RIGHT question!)
This is actually the trickiest part to understand. Let me explain with a concrete example:
Imagine your IMU is tilted 45 degrees:
        IMU Frame (tilted)
            ↑ z
           /
          /
         /___→ x


    Global Frame (level with ground)
         ↑ z
         |
         |___→ x
Now, you accelerate forward (in global x-direction):

Global frame sees: acceleration = [1, 0, 0] (moving in x)
IMU sees: acceleration = [0.7, 0, 0.7] (because it's tilted, it feels acceleration in BOTH its x and z axes!)

The key insight: The IMU always reports acceleration in its own tilted coordinate frame, not the global frame.
As you integrate and the sensor rotates more, the IMU frame keeps changing. So you need to rotate each acceleration measurement into the global frame before adding them up.
Another way to think about it:

Step 1: IMU tilted 0° reads acc = [1, 0, 0] in its frame
Step 2: IMU tilted 10° reads acc = [1, 0, 0] in its frame
If you just add them: [1,0,0] + [1,0,0] = [2,0,0] ← WRONG! They were pointing slightly different directions!
You need to rotate each to global frame first, THEN add them.

Does this make more sense now?

2. Gravity Subtraction (You're PARTIALLY right!)
You're thinking like a real robotics engineer! Let me clarify:
You're RIGHT that:

Gravity varies by location (9.78 at equator, 9.83 at poles)
IMU precision is high (often 0.001 m/s² resolution)
Gravity can be used for alignment

But here's why we subtract it for preintegration:
The IMU accelerometer measures specific force, not true acceleration. When sitting still on a table:

True acceleration = 0 (not moving)
IMU reads = [0, 0, 9.81] (feels the table pushing up against gravity)

When you're moving:

IMU reads = true_acceleration + gravity_effect
To get true_acceleration = IMU_reading - gravity

Your concern about precision is valid but:

For short-term integration (50ms), using 9.81 is fine
The error from using 9.81 instead of 9.806 for 50ms is: 0.006 * 0.05 = 0.0003 m ← negligible!
For longer integration (multiple seconds), yes, you'd want accurate gravity

About using gravity for alignment:
You're absolutely right! In a full SLAM system, you would:

Use gravity direction to estimate initial IMU orientation (pitch/roll)
THEN do preintegration with gravity removed

For this calibration project, we'll assume the IMU is already roughly aligned.
So the answer: Subtract gravity for preintegration, but you're right that gravity is useful for initial alignment!


we already have the TF between all?EXCELLENT CATCH! You're thinking ahead!
Yes, you're absolutely right that we have extrinsic calibrations (transforms) between sensors. But here's the subtle distinction:
Two Different Transforms
1. Extrinsic Transform (Static):
T_imu_to_camera (or T_imu_to_lidar)

This is FIXED, never changes
Tells you where IMU sits relative to camera/lidar on the robot
Example: "IMU is 10cm behind camera, rotated 5° down"

2. Motion Transform (Dynamic - what we're computing):
T_motion = change from time t to time t+Δt

This CHANGES every moment
Tells you how the ENTIRE robot moved
Example: "Between t and t+Δt, the whole robot rotated 2° and moved 5cm forward"

Why We Still Need to Rotate Acceleration
Look at the paper's equation (Section III.A):
T = Te × Tv
Where:

Te = extrinsic calibration (static, what you're thinking of)
Tv = vehicle motion estimate (dynamic, what IMU preintegration gives you)

The IMU preintegration is computing Tv - how did the vehicle body (carrying ALL sensors) move?
Even though we know Te (where IMU sits on the robot), we still need to:

Read IMU accelerations in IMU frame
Rotate them to vehicle body frame as we integrate
This gives us Tv (how the body moved)

Concrete Example
t=0: Robot at position [0, 0, 0], facing north
     IMU reads acc = [1, 0, 0] in its own frame

t=0.01s: Robot rotated 10° clockwise
         IMU reads acc = [1, 0, 0] in its own frame (still!)
         
t=0.02s: Robot rotated 20° clockwise
         IMU reads acc = [1, 0, 0] in its own frame (still!)
Even though the IMU always reports in its own frame, the robot is rotating underneath it. So you need to account for that rotation when integrating.
The extrinsic Te doesn't help here because:

Te tells you where IMU sits on the robot (static offset)
It doesn't tell you how the robot rotated between t and t+Δt
That rotation is what we're COMPUTING with preintegration

So You're Right AND Wrong

✅ Right: We have extrinsic transforms
✅ Right: We'll use those later in the full calibration
❌ Wrong: They don't replace the need to rotate during preintegration

The extrinsic Te is used AFTER preintegration to transform between sensor coordinate frames. The preintegration computes motion in the body frame first.